# Mean filter

In [14]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from typing import Union, List, Tuple
import argparse

class MeanFilter:
    """Implementation of mean filtering for image processing."""
    
    def __init__(self, kernel_size: Union[int, Tuple[int, int]] = 3):
        # Convert to tuple if integer
        if isinstance(kernel_size, int):
            self.kernel_size = (kernel_size, kernel_size)
        else:
            self.kernel_size = kernel_size
            
        # Validate kernel dimensions (must be odd numbers)
        if self.kernel_size[0] % 2 == 0 or self.kernel_size[1] % 2 == 0:
            raise ValueError("Kernel dimensions must be odd numbers")
        
        # Create normalized kernel
        self.kernel = np.ones(self.kernel_size) / (self.kernel_size[0] * self.kernel_size[1])
        
    def apply(self, image: np.ndarray, border_type: int = cv2.BORDER_REFLECT) -> np.ndarray:
        # Apply 2D filter
        result = cv2.filter2D(image, -1, self.kernel, borderType=border_type)
        return result
    
    def apply_separable(self, image: np.ndarray, border_type: int = cv2.BORDER_REFLECT) -> np.ndarray:
        # Create 1D kernels for separable filtering
        kernel_x = np.ones((1, self.kernel_size[1])) / self.kernel_size[1]
        kernel_y = np.ones((self.kernel_size[0], 1)) / self.kernel_size[0]
        
        # Apply horizontal filter
        temp = cv2.filter2D(image, -1, kernel_x, borderType=border_type)
        
        # Apply vertical filter
        result = cv2.filter2D(temp, -1, kernel_y, borderType=border_type)
        
        return result
    
    def apply_builtin(self, image: np.ndarray) -> np.ndarray:
        return cv2.blur(image, self.kernel_size)

class ImageProcessor:
    """Class for loading, processing, and saving images."""
    
    @staticmethod
    def load_image(path: str, grayscale: bool = False) -> np.ndarray:
        if grayscale:
            img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        else:
            img = cv2.imread(path)
            if img is not None:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                
        if img is None:
            raise ValueError(f"Could not load image from {path}")
            
        return img
    
    @staticmethod
    def save_image(image: np.ndarray, path: str, is_rgb: bool = True) -> None:
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(os.path.abspath(path)), exist_ok=True)
        
        # Convert RGB to BGR if needed
        if is_rgb and len(image.shape) == 3 and image.shape[2] == 3:
            save_image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        else:
            save_image = image
            
        cv2.imwrite(path, save_image)

class FilterEvaluator:
    """Class for evaluating and comparing filter performance."""
    
    @staticmethod
    def psnr(original: np.ndarray, filtered: np.ndarray) -> float:
        mse = np.mean((original.astype(float) - filtered.astype(float)) ** 2)
        if mse == 0:
            return float('inf')
        
        max_pixel = 255.0
        return 20 * np.log10(max_pixel / np.sqrt(mse))
    
    @staticmethod
    def visualize_comparison(images: List[np.ndarray], titles: List[str], 
                            figsize: Tuple[int, int] = (12, 8)) -> None:
        assert len(images) == len(titles), "Number of images must match number of titles"
        
        num_images = len(images)
        plt.figure(figsize=figsize)
        
        for i in range(num_images):
            plt.subplot(1, num_images, i + 1)
            plt.title(titles[i])
            
            if len(images[i].shape) == 2:  # Grayscale
                plt.imshow(images[i], cmap='gray')
            else:  # Color
                plt.imshow(images[i])
                
            plt.axis('off')
            
        plt.tight_layout()
        plt.show()


def main():
    # Parse command-line arguments
    parser = argparse.ArgumentParser(description="Apply mean filtering to an image.")
    parser.add_argument("--image-path", type=str, default="input.jpg", 
                        help="input/part_a_filtering/gaussian.png")
    args = parser.parse_args()
    
    try:
        # Load image
        processor = ImageProcessor()
        original_image = processor.load_image(args.image_path, grayscale=False)
        
        # Apply mean filters with different kernel sizes and methods
        mean_filter_3x3 = MeanFilter(3)
        
        # Apply filters
        filtered_3x3 = mean_filter_3x3.apply(original_image)
        filtered_3x3_separable = mean_filter_3x3.apply_separable(original_image)
        filtered_3x3_builtin = mean_filter_3x3.apply_builtin(original_image)
        
        # Compare results
        evaluator = FilterEvaluator()
        psnr_3x3 = evaluator.psnr(original_image, filtered_3x3)
        psnr_3x3_separable = evaluator.psnr(original_image, filtered_3x3_separable)
        psnr_3x3_builtin = evaluator.psnr(original_image, filtered_3x3_builtin)
        
        print(f"PSNR with 3x3 kernel (filter2D): {psnr_3x3:.2f} dB")
        print(f"PSNR with 3x3 kernel (separable): {psnr_3x3_separable:.2f} dB")
        print(f"PSNR with 3x3 kernel (builtin): {psnr_3x3_builtin:.2f} dB")
        
        # Visualize results
        evaluator.visualize_comparison(
            [original_image, filtered_3x3, filtered_3x3_separable, filtered_3x3_builtin],
            ["Original", "Noisy", "Mean 3x3 (filter2D)", "Mean 3x3 (Separable)", "Mean 3x3 (Builtin)"],
            figsize=(15, 5)
        )
        
        # Save results
        output_dir = "filter_results"
        processor.save_image(original_image, f"{output_dir}/original.png")
        processor.save_image(filtered_3x3, f"{output_dir}/mean_3x3.png")
        processor.save_image(filtered_3x3_separable, f"{output_dir}/mean_3x3_separable.png")
        processor.save_image(filtered_3x3_builtin, f"{output_dir}/mean_3x3_builtin.png")
        
        print(f"Results saved to {output_dir} directory")
        
    except ValueError as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    main()

usage: ipykernel_launcher.py [-h] [--image-path IMAGE_PATH]
ipykernel_launcher.py: error: unrecognized arguments: --f=/run/user/1000/jupyter/runtime/kernel-v3d4115bbc3472875b771ff68fe4e8d58286304796.json


SystemExit: 2

/home/dani/miniconda3/envs/AI/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3678: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
